In [1]:
#%pip install llama-index llama-index-vector-stores-upstash upstash-vector

In [2]:
import setup
setup.init_django()

In [3]:
django_pdf = "data/django.pdf"

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader , StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore

In [5]:
reader = SimpleDirectoryReader(input_files=[django_pdf])

In [6]:
import helpers
import openai
openai.api_key = helpers.config("OPEANAI_API_KEY", default=None)

In [7]:
url = helpers.config("UPSTASH_VECTOR_URL", default=None)
token = helpers.config("UPSTASH_VECTOR_TOKEN", default=None)

print(token)

ABYFMHN1bW1hcnktaWJleC01Njg4MS1ldTFhZG1pbll6azBaRGN5WW1JdE1HTmxOUzAwTnpWaUxUZ3lNamd0TURNNVptRXpNVFV4WldZeA==


In [8]:
data = reader.load_data()

In [9]:
import httpx
timeout = httpx.Timeout(300.0,  connect=300.0)
client = httpx.Client(timeout=timeout)

In [10]:
from upstash_vector import Index

upstash_index = Index(
    url=url, 
    token=token,
    retries=5,
    retry_interval=0.2
)
upstash_index._client = client


In [11]:
vector_store = UpstashVectorStore(
    url=url, 
    token=token,
)
vector_store._index = upstash_index


In [12]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [13]:
index = VectorStoreIndex.from_documents(data, storage_context=storage_context, insert_batch_size=500)

UpstashError: Exceeded daily write limit: 10000

In [ ]:
index_persist = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [30]:
query_engine = index_persist.as_query_engine()

In [33]:
r = query_engine.query("What are Signals")
print(r.response)

Signals in Django are a set of messages sent by the model system to notify certain actions or events. They allow decoupled applications to get notified when certain actions occur elsewhere in the application. Signals can be used to perform additional logic or trigger specific functions in response to events like model instantiation, saving, or deletion.


In [ ]:
# index = VectorStoreIndex.from_documents(data)

In [ ]:
# query_engine = index.as_query_engine()

In [ ]:
# r =  query_engine.query("Explain to me signals concept")
# print(r.response)